In [11]:
from bs4 import BeautifulSoup as bs
import requests
import time, os
import re
import pandas as pd 
import pickle
import dateutil.parser



In [12]:
df = pd.read_csv('movie_cleaned.csv')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1629 entries, 0 to 1628
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           1629 non-null   object
 1   year            1629 non-null   int64 
 2   domestic_gross  1629 non-null   int64 
 3   foreign_gross   1629 non-null   int64 
 4   total_gross     1629 non-null   int64 
 5   budget          1629 non-null   int64 
 6   opening         1629 non-null   int64 
 7   distributor     1629 non-null   object
 8   mpaa            1629 non-null   object
 9   runtime         1629 non-null   int64 
 10  genre           1629 non-null   object
 11  director        1629 non-null   object
 12  star            1629 non-null   object
dtypes: int64(7), object(6)
memory usage: 165.6+ KB


In [16]:
m = df[df['opening'] == 0]
m

,title,year,domestic_gross,foreign_gross,total_gross,budget,opening,distributor,mpaa,runtime,genre,director,star
470,Sea Rex 3D: Journey to a Prehistoric World,2010,6096582,9907568,16004150,5000000,0,3D Entertainment,Not Rated,41,Documentary,0,0
1183,The Lady in the Van,2015,10021175,31366512,41387687,6000000,0,Sony Pictures Classics,PG-13,104,Biography,Nicholas Hytner,Maggie Smith
1203,Batman: The Killing Joke,2016,3775000,687034,4462034,3500000,0,Fathom Events,R,76,Action,Sam Liu,Kevin Conroy


In [19]:
m = df[df['runtime'] == 0]
m

,title,year,domestic_gross,foreign_gross,total_gross,budget,opening,distributor,mpaa,runtime,genre,director,star


In [ ]:
pipeline_lst = []
url_base = 'https://www.boxofficemojo.com/year/{}/'
url_list = [url_base.format(i) for i in range(2009,2020)]
for url in url_list:
    page = requests.get(url).text
    soup = bs(page,'lxml')
    for h in soup.find_all('a', href = re.compile('/release')): 
        link = h['href']
        base = 'https://www.boxofficemojo.com'
        
        base +=link
        pipeline_lst.append(base)
        

print(len(pipeline_lst))


In [ ]:
def money_int(moneystr):
    moneystr = moneystr.replace('$', '').replace(',', '')
    return int(moneystr)

def runtime_minutes(runtimestr):
    s= runtimestr.split()
    if len(s) == 4: 

        minutes = int(s[0])*60 + int(s[2])
        return minutes
    
    else: 
        if len(s) == 2: 
            if 'hr' in s: 
 
                minutes = int(s[0])*60
                return minutes

            if 'min' in s:

                minutes = s[0]
                return minutes 
               


In [ ]:
#scrape into a dict
    
title = []
year = []
domestic_gross = []
foreign_gross = []
total_gross = []
budget = []
opening = []
distributor=[]
mpaa=[]
runtime=[]
genre=[]

    
    
for a in pipeline_lst: 
    page = requests.get(a).text
    soup = bs(page,'lxml')
      
    headers = ['title', 'year', 'domestic_gross', 'foreign_gross',
               'total_gross', 'budget','opening'
             ,'distributor', 'mpaa', 'runtime', 'genre']

        #scrape title 
    try: 
        title_tag = soup.find('title').text
        m_title = title_tag.split('-')[0].strip()
        title.append(m_title)
    except: 
        m_title = ''
        title.append(m_title)

        #scrape year
    try: 
        m_year= (soup.find('span', text = re.compile('Release Date'))
                        .find_next('a').text.split(',')[1].strip())
        year.append(m_year)
    except:
        m_year= ''
        year.append(m_year)
        
        #scrape domestic gross  
    try: 
        domestic = (soup.find('span', class_='a-size-small')
            .find_next('span', class_="a-size-medium a-text-bold")
            .find('span', class_='money').text)

        domestic_m = money_int(domestic)
        domestic_gross.append(domestic_m)
    except: 
        domestic_m = ''
        domestic_gross.append(domestic_m)

        #scrape international gross
    try: 
        int_gross_tag = (soup.find('span', class_="a-size-small")
                        .find_next('span', class_='a-size-small')
                        .find_next('span', class_="a-size-medium a-text-bold")
                        .find('span', class_='money').text)
        foreign_m = money_int(int_gross_tag)
        foreign_gross.append(foreign_m)
    except: 
        foreign_m = ''
        foreign_gross.append(foreign_m)

            #scrape total gross 
    try: 
        world_gross_tag = (soup.find('span', class_="a-size-small")
                          .find_next('span', class_='a-size-small')
                          .find_next('span', class_='a-size-small')
                          .find_next('span', class_="a-size-medium a-text-bold")
                          .find('span', class_='money').text)

        world_m= money_int(world_gross_tag)
        total_gross.append(world_m)
    except: 
        world_m = ''
        total_gross.append(world_m)
        
        #scrape budget 
    try:
        budget_tag = (soup.find('span', text = re.compile('Budget'))
                     .find_next('span', class_="money").text)
        
        budget_m= money_int(budget_tag)
        budget.append(budget_m)
    except: 
        budget_m = ''
        budget.append(budget_m)
    
    #scrape opening
    try:
        opening_tag = (soup.find('span', text= re.compile('Opening'))
                      .find_next('span',class_="money").text)
        
        opening_m = money_int(opening_tag)
        opening.append(opening_m)
    except: 
        opening_m = ''
        opening.append(opening_m) 

        #scrape distributor 
    try: 
        distributor_m = (soup.find('span', text = 'Distributor')
                               .find_next().text.replace('See full company information\n\n', ''))
        distributor.append(distributor_m)
    except: 
        distributor_m = ''       
        distributor.append(distributor_m)
      

    # scrape MPAA
    try: 
        rating_m =(soup.find('span', text = 'MPAA')
                        .find_next('span').text)
        mpaa.append(rating_m)
    except: 
        rating_m = ''
        mpaa.append(rating_m)


                # scrape running time 
    try: 
        running_time =(soup.find('span', text = 'Running Time')
                    .find_next('span').text)
        
        runtime_m= runtime_minutes(running_time)
        
        runtime.append(runtime_m)
        
    except: 
        runtime_m= ''
        runtime.append(runtime_m)

                #scrape Genre
    try: 
        genre_m =(soup.find('span', text = 'Genres')
                    .find_next('span').text.split()[0])
        genre.append(genre_m)
    except: 
        genre_m= ''

        genre.append(genre_m)
       
    movie_dict = dict(zip(headers, [title,year, domestic_gross, foreign_gross, 
                                       total_gross, budget,opening, distributor, mpaa,
                                      runtime, genre]))

    


In [ ]:
movie_dict

In [ ]:
df = pd.DataFrame(movie_dict)
df.head()

In [ ]:
#save df to pickle 
import pickle

df.to_pickle('movie_no_clean.pkl')


In [ ]:
#read pickle file as dataframe 

movie_df = pd.read_pickle('movie_no_clean.pkl')

In [ ]:
movie_df

# Clean Data

In [ ]:
movie_df.dtypes

In [ ]:
movie_df.info()

In [ ]:
#Nan values to 0

import numpy as np
movie_df.replace(np.nan, 0, inplace = True)

movie_df.info()


In [ ]:
#check if any values are null 
movie_df.isnull().values.any()


In [ ]:
movie_df.replace('', 0, inplace = True)

In [ ]:
#replace runtime as int 

movie_df['runtime']= movie_df['runtime'].astype('int64')



movie_df.dtypes

In [ ]:
movie_df.head()

In [ ]:
movie_df.dtypes

In [ ]:
movie_df.shape

In [ ]:
#drop for duplicates
movie_df = movie_df.drop_duplicates(subset=['title','year', 'runtime'])

movie_df.shape



In [ ]:
# check unique values for distributor 

movie_df['distributor'].value_counts()

In [ ]:
#check if any movies have no distributor 

mask = movie_df[movie_df['distributor'] == 0]
mask

In [ ]:
#drop distributor where it is 0 
movie_df.drop(movie_df.index[movie_df['distributor'] == 0], inplace = True)
movie_df.shape

In [ ]:
# check unique values for mpaa
movie_df['mpaa'].value_counts()

In [ ]:
# replace 0 as not rated 
movie_df['mpaa'].replace(0,'Not Rated', inplace =True, regex = True)
movie_df['mpaa'].value_counts()

In [ ]:
# check unique values for genre

movie_df['genre'].value_counts()

In [ ]:
#drop genre where it is 0

movie_df.drop(movie_df.index[movie_df['genre']==0], inplace = True)
movie_df.shape

In [ ]:
#look into runtime

movie_df['runtime'].value_counts()

In [ ]:
#check if any movies have no runtime

mask = movie_df[movie_df['runtime'] == 0]
mask

In [ ]:
#drop runtime where it is 0

movie_df.drop(movie_df.index[movie_df['runtime']==0], inplace = True)
movie_df.shape

In [ ]:
#check if any movies have no profit and no budget

mask = movie_df[ (movie_df['budget'] == 0) ] 
mask




In [ ]:
movie_df.shape

In [ ]:
movie_df.info()

In [ ]:
#save webscraped csv 

movie_df.to_csv('webscraped.csv', index = False)